## Chargement du dataset et création d'un dataset d'entraînement composé d'un dico avec 2 clés : text et label

In [19]:
import csv
import pandas as pd

file_path_train = "./data/train_submission.csv"
file_path_test = "./data/test_without_labels.csv"

data_train = pd.read_csv(file_path_train)


In [20]:
data_train

,ID,Usage,Text,Label
0,136,Public,Finalment Atena le recibe en l'acropoli d'Ate...,arg
1,62,Public,Jane Laffort fille de Joseph Laffort et d' Ang...,lat
2,74,Public,Сонзэ ялаксонзо - Роджер Джозеф Бошкович - у...,myv
3,40,Public,Mɛniɛ nkùɔ dìì mɔ̀nnì bɛnìtìbɛ̀ kɛ́deè kɛ̀ Nɔ...,tbz
4,30,Public,Ka go dirisa thekniki yeo ya phetogonepiso Le...,tsn
...,...,...,...,...
38849,66,Public,Gorillas es divide in duo species e 4 o 5 subs...,ina
38850,62,Public,Bakebi bamba se nansha nanku etshi mukila wa ...,lua
38851,8,Public,All Äonen de dorvör liggen doot weern fröher...,nds
38852,118,Public,اور اگر یہ لوگ اپنی جانوں پر ظلم کر کے آپکے حض...,urd


## Analyse of the data train

In [21]:
data_train_without_label = data_train[data_train["Label"].isna()]

In [22]:
data_train_without_label

,ID,Usage,Text,Label
1273,26,Public,Āu-lâi in-ūi goân-chū-bîn ê kong-kek 1541 nî ...,NaN
1697,68,Public,Chrzowice sī chi̍t ê tī Pho-lân Kiōng-hô-kok O...,NaN
1976,116,Public,Tī pún só͘-chāi sì-ûi ê tē-hng ū Drávaszerdahe...,NaN
2870,4,Public,Darahanove (Ukraina-gí: ) sī chi̍t ê tī Ukrain...,NaN
3079,46,Public,Chit ūi tī 2010 nî ê jîn-kháu-sò͘ sī 1 113 lâng.,NaN
...,...,...,...,...
36941,6,Public,Arandon sī ūi-tī Hoat-kok Rhône-Alpes toā-khu ...,NaN
37066,126,Public,Haft Cheshmeh-ye Jahanshah (Pho-su-gí: ) sī ch...,NaN
37487,26,Public,Bulair (Bulgaria-gí: ) sī chi̍t ê tī Bulgaria ...,NaN
38375,108,Public,Bô phah-sǹg tī sin-le̍k 10 go̍eh 29 hō ē-po͘ ...,NaN


Il y a 100 instances qui ne sont pas labellisées. 

In [23]:
data_train_without_nan_for_label = data_train.dropna()

In [24]:
number_of_languages = len(data_train["Label"].unique())
print(f"Il y a {number_of_languages} différentes langues dans le dataset de train")

Il y a 390 différentes langues dans le dataset de train


In [25]:
len(data_train["ID"].unique())

80

On observe qu'une langue n'est pas identifiée par son ID car si c'était le cas on aurait le même nombre d'ID différents que de langues différentes. La colonne ID ne semble ainsi nous apporter aucune information

### Analyse stats sur les données labellisées

In [26]:
dataset_sorted_by_number_instances_by_language = data_train_without_nan_for_label.groupby("Label").count().sort_values('Usage', ascending=False)
dataset_sorted_by_number_instances_by_language

,ID,Usage,Text
Label,,,
tgk,300,300,300
guj,200,200,200
tat,200,200,200
crh,200,200,200
kaa,200,200,200
...,...,...,...
gil,2,2,2
toi,1,1,1
kua,1,1,1


On observer que le nombre d'exemples par langue varie énormément. Certaines langues sont sur-représentées (avec 300 instances pour la première) par rapport à d'autres. 

In [27]:
percentage_of_languages_with_at_least_100_instances = len(dataset_sorted_by_number_instances_by_language[dataset_sorted_by_number_instances_by_language["ID"] >= 100])/len(dataset_sorted_by_number_instances_by_language) * 100
print(f"Le pourcentage de langues avec au moins 100 instances est {percentage_of_languages_with_at_least_100_instances}%")

Le pourcentage de langues avec au moins 100 instances est 93.31619537275064%


## Pré-traitement du dataset de train

In [28]:
import string
import re 

def cleaning(text): 
    """
    Fonction pour pré-traiter le texte en enlevant tous les éléments de ponctuation, les chiffres et les double espaces. 
    """
    text_without_dash = text.replace('-', ' ')
    text_without_punctuation = text_without_dash.translate(str.maketrans('', '', string.punctuation))
    text_without_punctuation_figures = text_without_punctuation.translate(str.maketrans('','', string.digits))
    text_cleaned = re.sub(r'\s+', ' ', text_without_punctuation_figures).strip()

    return(text_cleaned)


In [29]:
import nltk
from nltk.corpus import words

# Télécharger la liste des mots en anglais (une seule fois nécessaire)
nltk.download('words')

# Liste des mots en anglais
english_words = set(words.words())


[nltk_data] Downloading package words to
[nltk_data]     /Users/hippolytelecomte/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [30]:
def remove_most_english_words(text): 
    """
    Fonction pour enlever les mots anglais lorsque la langue du texte n'est pas l'anglais. 
    """
    english_words = set(words.words())
    tokens = text.split() 
    filtered_tokens = [word for word in tokens if word.lower() not in english_words]

    return ' '.join(filtered_tokens)

Séparation entre le train et le val

In [31]:
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(data_train_without_nan_for_label, test_size=0.2, random_state=42)

In [32]:
train_set

,ID,Usage,Text,Label
7671,58,Public,شەکیلا یەکەم دەرکەوتنی ئەلبومی بە ناونیشانی کا...,ckb
5849,68,Public,Эркектердин тойодо сетка которгону уят ал эми...,kir
31089,10,Public,What does the first head represent and why ?,hin
10535,24,Public,Kaya pala mga Pinoy sobrang emotional. Mind c...,fil
30413,76,Public,Mogao bih jebati bukvalno bilo koga.,bos
...,...,...,...,...
6280,24,Public,Tañi feypipeelchi dungu ta tüfa : kiñe pichich...,arn
11311,50,Public,(65438) 2002 TA280 adolah sabuah asteroid nan ...,min
38256,122,Public,Taita Diospaj huasipica Jesustami ñucanchipa...,qvi
860,66,Public,Lemmopsis pelodes je porost co go nojprzōd ôp...,szl


Application du pré-traitement sur tout le dataframe

In [33]:
from tqdm import tqdm
tqdm.pandas()  

def pre_processing(df): 
    df['Text'] = df['Text'].apply(cleaning)
    
    df['Text'] = df.progress_apply(
        lambda row: remove_most_english_words(row['Text']) if row['Label'] != 'eng' else row['Text'], axis=1
    )

    df['Text'] = df['Text'].str.replace(' ', '', regex=False)
    
    return df


In [34]:
train_set = pre_processing(train_set)
val_set = pre_processing(val_set)

100%|██████████| 7751/7751 [08:39<00:00, 14.93it/s]


In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()
x_train = train_set['Text'].tolist()
y_train = train_set['Label'].tolist()
x_val = val_set['Text'].tolist()
y_val = val_set['Label'].tolist()

x_train = vectorizer.fit_transform(x_train)
x_val = vectorizer.transform(x_val)

In [36]:
naive_bayes = MultinomialNB(alpha= 0.0001, fit_prior = False) 
naive_bayes.fit(x_train,y_train)

MultinomialNB(alpha=0.0001, fit_prior=False)

In [37]:
from sklearn.metrics import accuracy_score, classification_report

predictions = naive_bayes.predict(x_val)
accuracy = accuracy_score(y_val, predictions)
print("Accuracy:", accuracy)

# Classification report (en complément)
print("Classification Report:\n", classification_report(y_val, predictions))

Accuracy: 0.09572958327957683
Classification Report:
               precision    recall  f1-score   support

         abk       0.00      1.00      0.01        25
         ace       0.00      0.00      0.00        16
         ach       0.00      0.00      0.00        17
         acm       0.00      0.00      0.00        20
         acr       1.00      0.05      0.09        22
         ada       0.00      0.00      0.00         1
         afb       0.00      0.00      0.00        31
         afr       0.00      0.00      0.00        23
         ahk       0.00      0.00      0.00        20
         ajp       0.00      0.00      0.00        15
         aka       0.00      0.00      0.00        21
         aln       0.00      0.00      0.00        21
         als       0.00      0.00      0.00        24
         alt       0.00      0.00      0.00        21
         amh       0.00      0.00      0.00        27
         aoj       0.00      0.00      0.00        23
         apc       0.00    

/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hippolytelecomte/Documents/Etudes/CentraleSupelec/3A/ANLP/TD/myenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sample

Il faut supprimer toute la ponctuation, les chiffres, les parenthèses, les liens, les double espaces